In [ ]:
import glob
import tqdm
import random
import pandas as pd
import numpy as np

In [ ]:
from scipy import stats

In [ ]:
from sklearn.metrics import ndcg_score, dcg_score, average_precision_score

In [ ]:
dfis = []
for fname in glob.glob('/Users/prashantbudania/Downloads/1bquxjob_day*'):
    df_temp = pd.read_csv(fname)
    df_temp['time_read'] = df_temp['time_read'].fillna(0)
    dfis.append(df_temp)

In [ ]:
len(dfis)

In [ ]:
TIME_READ = 5

In [ ]:
df_final = pd.concat(dfis, axis=0)

In [ ]:
bm25_users_daily = []
tfidf_users_daily = []
bm25_dfs = []

for dfi in dfis:
    df_bm25_temp = dfi[dfi['is_bm25_active']==True]
    bm25_users = set(df_bm25_temp['user_id'].unique().tolist())
    
    df_tfidf_temp = dfi[dfi['is_bm25_active']==False]
    tfidf_users = [user for user in df_tfidf_temp['user_id'].unique().tolist() if user not in bm25_users]
    
    bm25_users_daily.append(bm25_users)
    tfidf_users_daily.append(tfidf_users)
    
    bm25_dfs.append(df_bm25_temp)

In [ ]:
sum([len(x) for x in bm25_users_daily])

In [ ]:
df_final['time_read'] = df_final['time_read'].fillna(0)

In [ ]:
df_final['PageRead'] = df_final['PageRead'].fillna(1.5)

In [ ]:
df_final['timeReadActive'] = df_final.apply(lambda x: x['time_read']/((x['PageRead'] or 1.5)**0.5), axis=1)

In [ ]:
df_final['normalized_time_score'] = df_final.apply(lambda x: x['timeReadActive']/(x['timeReadActive']+25), axis=1)

In [ ]:
df_final = df_final.fillna(0)

In [ ]:
df_final['final_score'] = df_final.apply(lambda x: 25*(x['normalized_time_score']+int(x['Download']>0)+int(x['CreateAnnotation']>0)+int(x['Print']>0)+int(x['CopyGeneratedDocUrl']>0)+int(x['DocumentBookmarking']>0)+int(x['CustomTagsUpdate']>0))+2*(int(x['StatementSelect']>0)+int(x['SearchWithinDocument']>0)) if (x['timeReadActive']>=5 or (int(x['Download']>0)+int(x['CreateAnnotation']>0)+int(x['Print']>0)+int(x['CopyGeneratedDocUrl']>0)+int(x['DocumentBookmarking']>0)+int(x['CustomTagsUpdate']>0))>0) else -1.0, axis=1)

In [ ]:
df_final['gold_labels'] = df_final.apply(lambda x: 2 if x['final_score']>=10 else (1 if x['final_score']>0 else 0), axis=1)

In [ ]:
df_final['is_relevant'] = df_final.apply(lambda x: 1 if x['final_score']>=10 else (1 if x['final_score']>0 else 0), axis=1)

In [ ]:
df_bm25 = df_final[df_final['is_bm25_active']==True]

In [ ]:
df_tfidf = df_final[df_final['is_bm25_active']==False]

In [ ]:
df_bm25.shape, df_tfidf.shape

In [ ]:
BUCKET_MAP = {'1': 0, '2-3': 1, '4-5': 2, '6-10': 3, '11-20': 4, '21-inf': 5, 'ALL': 6}

In [ ]:
bm25_sessions = [[],[],[],[],[],[],[]]
bm25_document_clicks = [[],[],[],[],[],[],[]]
bm25_median_time_read = [[],[],[],[],[],[],[]]
bm25_unique_users = [[],[],[],[],[],[],[]]
bm25_avg_rank = [[],[],[],[],[],[],[]]
bm25_doc_clicks_with_dwell_time_less_than_5sec = [[],[],[],[],[],[],[]]

bm25_percentage_clicks_in_top3_leading_to_insights = [[],[],[],[],[],[],[]]
bm25_percentage_clicks_in_top5_leading_to_insights = [[],[],[],[],[],[],[]]
bm25_percentage_clicks_in_top10_leading_to_insights = [[],[],[],[],[],[],[]]
bm25_percentage_clicks_in_top20_leading_to_insights = [[],[],[],[],[],[],[]]

bm25_percentage_clicks_in_top3 = [[],[],[],[],[],[],[]]
bm25_percentage_clicks_in_top5 = [[],[],[],[],[],[],[]]
bm25_percentage_clicks_in_top10 = [[],[],[],[],[],[],[]]
bm25_percentage_clicks_in_top20 = [[],[],[],[],[],[],[]]

bm25_ndcg_5 = [[],[],[],[],[],[],[]]
bm25_ndcg_10 = [[],[],[],[],[],[],[]]
bm25_ndcg_20 = [[],[],[],[],[],[],[]]

bm25_average_precision = [[],[],[],[],[],[],[]]

bm25_total_insights = 0
bm25_clicks_to_insights = []

for search_session in df_bm25['search_context'].unique().tolist():
    
    df_temp = df_bm25[df_bm25['search_context']==search_session]
    NUM = df_temp.shape[0]
    
    df_insights = df_temp[df_temp['time_read']>=5]
    INSIGHTS = df_insights.shape[0]
    bm25_total_insights += INSIGHTS
    
    insight_positions = df_insights['position_for_score_desc'].tolist()
    click_positions = df_temp['position_for_score_desc'].tolist()
    if insight_positions:
        first_insight_position = min(insight_positions)
        for idx, pos in enumerate(click_positions):
            if pos == first_insight_position:
                bm25_clicks_to_insights.append(idx)
                break
    
    if NUM == 1:
        bucket_id = '1'    
    
    elif NUM <= 3:
        bucket_id = '2-3'
        
    elif NUM <= 5:
        bucket_id = '4-5'
        
    elif NUM <= 10:
        bucket_id = '6-10'
        
    elif NUM <= 20:
        bucket_id = '11-20'

    else:
        bucket_id = '21-inf'
    
    session = 1
    median_time_read = np.median(df_temp['time_read'].tolist())
    unique_users = df_temp['user_id'].unique().tolist()
    avg_rank = df_temp['position_for_score_desc'].sum()/NUM
    doc_clicks_with_dwell_time_less_than_5sec = df_temp[df_temp['time_read']<5].shape[0]/NUM
    
    unique_clicks3 = [x<=3 for x in list(set(df_temp['position_for_score_desc'].tolist()))]
    unique_clicks5 = [x<=5 for x in list(set(df_temp['position_for_score_desc'].tolist()))]
    unique_clicks10 = [x<=10 for x in list(set(df_temp['position_for_score_desc'].tolist()))]
    unique_clicks20 = [x<=20 for x in list(set(df_temp['position_for_score_desc'].tolist()))]
    
    unique_insights3 = [x<=3 for x in list(set(df_insights['position_for_score_desc'].tolist()))]
    unique_insights5 = [x<=5 for x in list(set(df_insights['position_for_score_desc'].tolist()))]
    unique_insights10 = [x<=10 for x in list(set(df_insights['position_for_score_desc'].tolist()))]
    unique_insights20 = [x<=20 for x in list(set(df_insights['position_for_score_desc'].tolist()))]
    
    percentage_clicks_in_top3 = sum(unique_clicks3)/min(len(unique_clicks3),3)
    percentage_clicks_in_top5 = sum(unique_clicks5)/min(len(unique_clicks5),5)
    percentage_clicks_in_top10 = sum(unique_clicks10)/min(len(unique_clicks10),10)
    percentage_clicks_in_top20 = sum(unique_clicks20)/min(len(unique_clicks20),20)
    
    percentage_clicks_in_top3_leading_to_insights = sum(unique_insights3)/min(len(unique_clicks3),3)
    percentage_clicks_in_top5_leading_to_insights = sum(unique_insights5)/min(len(unique_clicks5),5)
    percentage_clicks_in_top10_leading_to_insights = sum(unique_insights10)/min(len(unique_clicks10),10)
    percentage_clicks_in_top20_leading_to_insights = sum(unique_insights20)/min(len(unique_clicks20),20)

    
    average_precision = average_precision_score(df_temp['is_relevant'].tolist(), [1000-x for x in df_temp['position_for_score_desc'].tolist()])
    
    
    if df_temp['position_for_score_desc'].max() >= 2:
        temp = [0]*5
        for idx in range(df_temp.shape[0]):
            row = df_temp.iloc[idx,:]
            if row['position_for_score_desc'] <= 5:
                temp[row['position_for_score_desc']-1] = max(row['gold_labels'], temp[row['position_for_score_desc']-1])

        if sum(temp) != 0:
            bm25_ndcg_5[BUCKET_MAP[bucket_id]].append(temp)
            bm25_ndcg_5[BUCKET_MAP['ALL']].append(temp)

        temp = [0]*10
        for idx in range(df_temp.shape[0]):
            row = df_temp.iloc[idx,:]
            if row['position_for_score_desc'] <= 10:
                temp[row['position_for_score_desc']-1] = max(row['gold_labels'], temp[row['position_for_score_desc']-1])

        if sum(temp) != 0:
            bm25_ndcg_10[BUCKET_MAP[bucket_id]].append(temp)
            bm25_ndcg_10[BUCKET_MAP['ALL']].append(temp)

        temp = [0]*20
        for idx in range(df_temp.shape[0]):
            row = df_temp.iloc[idx,:]
            if row['position_for_score_desc'] <= 20:
                temp[row['position_for_score_desc']-1] = max(row['gold_labels'], temp[row['position_for_score_desc']-1])

        if sum(temp) != 0:
            bm25_ndcg_20[BUCKET_MAP[bucket_id]].append(temp)
            bm25_ndcg_20[BUCKET_MAP['ALL']].append(temp)
    
    
    for bucket in [bucket_id, 'ALL']:
    
        bm25_sessions[BUCKET_MAP[bucket]].append(session)
        bm25_document_clicks[BUCKET_MAP[bucket]].append(NUM)
        bm25_median_time_read[BUCKET_MAP[bucket]].append(median_time_read)
        bm25_unique_users[BUCKET_MAP[bucket]] += unique_users
        bm25_avg_rank[BUCKET_MAP[bucket]].append(avg_rank)
        bm25_doc_clicks_with_dwell_time_less_than_5sec[BUCKET_MAP[bucket]].append(doc_clicks_with_dwell_time_less_than_5sec)

        bm25_percentage_clicks_in_top3_leading_to_insights[BUCKET_MAP[bucket]].append(percentage_clicks_in_top3_leading_to_insights)
        bm25_percentage_clicks_in_top5_leading_to_insights[BUCKET_MAP[bucket]].append(percentage_clicks_in_top5_leading_to_insights)
        bm25_percentage_clicks_in_top10_leading_to_insights[BUCKET_MAP[bucket]].append(percentage_clicks_in_top10_leading_to_insights)
        bm25_percentage_clicks_in_top20_leading_to_insights[BUCKET_MAP[bucket]].append(percentage_clicks_in_top20_leading_to_insights)

        bm25_percentage_clicks_in_top3[BUCKET_MAP[bucket]].append(percentage_clicks_in_top3)
        bm25_percentage_clicks_in_top5[BUCKET_MAP[bucket]].append(percentage_clicks_in_top5)
        bm25_percentage_clicks_in_top10[BUCKET_MAP[bucket]].append(percentage_clicks_in_top10)
        bm25_percentage_clicks_in_top20[BUCKET_MAP[bucket]].append(percentage_clicks_in_top20)
        
        if not np.isnan(average_precision):
            bm25_average_precision[BUCKET_MAP[bucket]].append(average_precision)


In [ ]:
tfidf_sessions = [[],[],[],[],[],[],[]]
tfidf_document_clicks = [[],[],[],[],[],[],[]]
tfidf_median_time_read = [[],[],[],[],[],[],[]]
tfidf_unique_users = [[],[],[],[],[],[],[]]
tfidf_avg_rank = [[],[],[],[],[],[],[]]
tfidf_doc_clicks_with_dwell_time_less_than_5sec = [[],[],[],[],[],[],[]]

tfidf_percentage_clicks_in_top3 = [[],[],[],[],[],[],[]]
tfidf_percentage_clicks_in_top5 = [[],[],[],[],[],[],[]]
tfidf_percentage_clicks_in_top10 = [[],[],[],[],[],[],[]]
tfidf_percentage_clicks_in_top20 = [[],[],[],[],[],[],[]]

tfidf_percentage_clicks_in_top3_leading_to_insights = [[],[],[],[],[],[],[]]
tfidf_percentage_clicks_in_top5_leading_to_insights = [[],[],[],[],[],[],[]]
tfidf_percentage_clicks_in_top10_leading_to_insights = [[],[],[],[],[],[],[]]
tfidf_percentage_clicks_in_top20_leading_to_insights = [[],[],[],[],[],[],[]]


tfidf_ndcg_5 = [[],[],[],[],[],[],[]]
tfidf_ndcg_10 = [[],[],[],[],[],[],[]]
tfidf_ndcg_20 = [[],[],[],[],[],[],[]]

tfidf_average_precision = [[],[],[],[],[],[],[]]

tfidf_total_insights = 0
tfidf_clicks_to_insights = []


for search_session in df_tfidf['search_context'].unique().tolist():
    
    df_temp = df_tfidf[df_tfidf['search_context']==search_session]
    NUM = df_temp.shape[0]
    
    df_insights = df_temp[df_temp['time_read']>=5]
    INSIGHTS = df_insights.shape[0]
    tfidf_total_insights += INSIGHTS
    
    insight_positions = df_insights['position_for_score_desc'].tolist()
    click_positions = df_temp['position_for_score_desc'].tolist()
    if insight_positions:
        first_insight_position = min(insight_positions)
        for idx, pos in enumerate(click_positions):
            if pos == first_insight_position:
                tfidf_clicks_to_insights.append(idx)
                break
    
    if NUM == 1:
        bucket_id = '1'    
    
    elif NUM <= 3:
        bucket_id = '2-3'
        
    elif NUM <= 5:
        bucket_id = '4-5'
        
    elif NUM <= 10:
        bucket_id = '6-10'
        
    elif NUM <= 20:
        bucket_id = '11-20'

    else:
        bucket_id = '21-inf'
    
    session = 1
    median_time_read = np.median(df_temp['time_read'].tolist())
    unique_users = df_temp['user_id'].unique().tolist()
    avg_rank = df_temp['position_for_score_desc'].sum()/NUM
    doc_clicks_with_dwell_time_less_than_5sec = df_temp[df_temp['time_read']<5].shape[0]/NUM
    
    unique_clicks3 = [x<=3 for x in list(set(df_temp['position_for_score_desc'].tolist()))]
    unique_clicks5 = [x<=5 for x in list(set(df_temp['position_for_score_desc'].tolist()))]
    unique_clicks10 = [x<=10 for x in list(set(df_temp['position_for_score_desc'].tolist()))]
    unique_clicks20 = [x<=20 for x in list(set(df_temp['position_for_score_desc'].tolist()))]
    
    unique_insights3 = [x<=3 for x in list(set(df_insights['position_for_score_desc'].tolist()))]
    unique_insights5 = [x<=5 for x in list(set(df_insights['position_for_score_desc'].tolist()))]
    unique_insights10 = [x<=10 for x in list(set(df_insights['position_for_score_desc'].tolist()))]
    unique_insights20 = [x<=20 for x in list(set(df_insights['position_for_score_desc'].tolist()))]
    
    percentage_clicks_in_top3 = sum(unique_clicks3)/min(len(unique_clicks3),3)
    percentage_clicks_in_top5 = sum(unique_clicks5)/min(len(unique_clicks5),5)
    percentage_clicks_in_top10 = sum(unique_clicks10)/min(len(unique_clicks10),10)
    percentage_clicks_in_top20 = sum(unique_clicks20)/min(len(unique_clicks20),20)
    
    percentage_clicks_in_top3_leading_to_insights = sum(unique_insights3)/min(len(unique_clicks3),3)
    percentage_clicks_in_top5_leading_to_insights = sum(unique_insights5)/min(len(unique_clicks5),5)
    percentage_clicks_in_top10_leading_to_insights = sum(unique_insights10)/min(len(unique_clicks10),10)
    percentage_clicks_in_top20_leading_to_insights = sum(unique_insights20)/min(len(unique_clicks20),20)

    
    average_precision = average_precision_score(df_temp['is_relevant'].tolist(), [1000-x for x in df_temp['position_for_score_desc'].tolist()])
    
    
    if df_temp['position_for_score_desc'].max() >= 2:
        temp = [0]*5
        for idx in range(df_temp.shape[0]):
            row = df_temp.iloc[idx,:]
            if row['position_for_score_desc'] <= 5:
                temp[row['position_for_score_desc']-1] = max(row['gold_labels'], temp[row['position_for_score_desc']-1])

        if sum(temp) != 0:
            tfidf_ndcg_5[BUCKET_MAP[bucket_id]].append(temp)
            tfidf_ndcg_5[BUCKET_MAP['ALL']].append(temp)

        temp = [0]*10
        for idx in range(df_temp.shape[0]):
            row = df_temp.iloc[idx,:]
            if row['position_for_score_desc'] <= 10:
                temp[row['position_for_score_desc']-1] = max(row['gold_labels'], temp[row['position_for_score_desc']-1])

        if sum(temp) != 0:
            tfidf_ndcg_10[BUCKET_MAP[bucket_id]].append(temp)
            tfidf_ndcg_10[BUCKET_MAP['ALL']].append(temp)

        temp = [0]*20
        for idx in range(df_temp.shape[0]):
            row = df_temp.iloc[idx,:]
            if row['position_for_score_desc'] <= 20:
                temp[row['position_for_score_desc']-1] = max(row['gold_labels'], temp[row['position_for_score_desc']-1])

        if sum(temp) != 0:
            tfidf_ndcg_20[BUCKET_MAP[bucket_id]].append(temp)
            tfidf_ndcg_20[BUCKET_MAP['ALL']].append(temp)
    
    
    for bucket in [bucket_id, 'ALL']:
    
        tfidf_sessions[BUCKET_MAP[bucket]].append(session)
        tfidf_document_clicks[BUCKET_MAP[bucket]].append(NUM)
        tfidf_median_time_read[BUCKET_MAP[bucket]].append(median_time_read)
        tfidf_unique_users[BUCKET_MAP[bucket]] += unique_users
        tfidf_avg_rank[BUCKET_MAP[bucket]].append(avg_rank)
        tfidf_doc_clicks_with_dwell_time_less_than_5sec[BUCKET_MAP[bucket]].append(doc_clicks_with_dwell_time_less_than_5sec)

        tfidf_percentage_clicks_in_top3_leading_to_insights[BUCKET_MAP[bucket]].append(percentage_clicks_in_top3_leading_to_insights)
        tfidf_percentage_clicks_in_top5_leading_to_insights[BUCKET_MAP[bucket]].append(percentage_clicks_in_top5_leading_to_insights)
        tfidf_percentage_clicks_in_top10_leading_to_insights[BUCKET_MAP[bucket]].append(percentage_clicks_in_top10_leading_to_insights)
        tfidf_percentage_clicks_in_top20_leading_to_insights[BUCKET_MAP[bucket]].append(percentage_clicks_in_top20_leading_to_insights)

        tfidf_percentage_clicks_in_top3[BUCKET_MAP[bucket]].append(percentage_clicks_in_top3)
        tfidf_percentage_clicks_in_top5[BUCKET_MAP[bucket]].append(percentage_clicks_in_top5)
        tfidf_percentage_clicks_in_top10[BUCKET_MAP[bucket]].append(percentage_clicks_in_top10)
        tfidf_percentage_clicks_in_top20[BUCKET_MAP[bucket]].append(percentage_clicks_in_top20)
        
        if not np.isnan(average_precision):
            tfidf_average_precision[BUCKET_MAP[bucket]].append(average_precision)
    

In [ ]:
metric_scores = []

In [ ]:
metric_scores.append(['Total # of sessions']+[(sum(bm25_sessions[idx]), sum(tfidf_sessions[idx])) for idx in range(7)])

In [ ]:
metric_scores.append(['% of ALL sessions']+[(float('{:.3f}'.format(100*sum(bm25_sessions[idx])/sum(bm25_sessions[-1]))), float('{:.3f}'.format(100*sum(tfidf_sessions[idx])/sum(tfidf_sessions[-1])))) for idx in range(7)])

In [ ]:
metric_scores.append(['Total # of document clicks']+[(sum(bm25_document_clicks[idx]), sum(tfidf_document_clicks[idx])) for idx in range(7)])

In [ ]:
metric_scores.append(['% of ALL document clicks']+[(float('{:.3f}'.format(100*sum(bm25_document_clicks[idx])/sum(bm25_document_clicks[-1]))), float('{:.3f}'.format(100*sum(tfidf_document_clicks[idx])/sum(tfidf_document_clicks[-1])))) for idx in range(7)])

In [ ]:
metric_scores.append(['# of unique Users']+[(len(set(bm25_unique_users[idx])), len(set(tfidf_unique_users[idx]))) for idx in range(7)])

In [ ]:
metric_scores.append(['% of ALL Users']+[(float('{:.3f}'.format(100*len(set(bm25_unique_users[idx]))/len(set(bm25_unique_users[-1])))), float('{:.3f}'.format(100*len(set(tfidf_unique_users[idx]))/len(set(tfidf_unique_users[-1]))))) for idx in range(7)])

In [ ]:
metric_scores.append(['Median Time Read (sec)']+[(float('{:.3f}'.format(np.mean(bm25_median_time_read[idx]))), float('{:.3f}'.format(np.mean(tfidf_median_time_read[idx])))) for idx in range(7)])

In [ ]:
metric_scores.append(['', '', '', '', '', '', '', ''])

In [ ]:
metric_scores.append(['Avg Position/Rank']+[('{:.3f}'.format(np.mean(bm25_avg_rank[idx]))+'±'+'{:.3f}'.format(np.std(bm25_avg_rank[idx])), '{:.3f}'.format(np.mean(tfidf_avg_rank[idx]))+'±'+'{:.3f}'.format(np.std(tfidf_avg_rank[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_avg_rank[idx],bm25_avg_rank[idx],equal_var=False).pvalue))) for idx in range(7)])

In [ ]:
metric_scores.append(['% Clicks w/ dwell time < 5sec']+[('{:.3f}'.format(100*np.mean(bm25_doc_clicks_with_dwell_time_less_than_5sec[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_doc_clicks_with_dwell_time_less_than_5sec[idx])), '{:.3f}'.format(100*np.mean(tfidf_doc_clicks_with_dwell_time_less_than_5sec[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_doc_clicks_with_dwell_time_less_than_5sec[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_doc_clicks_with_dwell_time_less_than_5sec[idx],bm25_doc_clicks_with_dwell_time_less_than_5sec[idx],equal_var=False).pvalue))) for idx in range(7)])

In [ ]:
metric_scores.append(['', '', '', '', '', '', '', ''])

In [ ]:
metric_scores.append(['% All Clicks in Top 3 Positions']+[('{:.3f}'.format(100*np.mean(bm25_percentage_clicks_in_top3[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_percentage_clicks_in_top3[idx])), '{:.3f}'.format(100*np.mean(tfidf_percentage_clicks_in_top3[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_percentage_clicks_in_top3[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_percentage_clicks_in_top3[idx],bm25_percentage_clicks_in_top3[idx],equal_var=False).pvalue))) for idx in range(7)])

In [ ]:
metric_scores.append(['% All Clicks in Top 5 Positions']+[('{:.3f}'.format(100*np.mean(bm25_percentage_clicks_in_top5[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_percentage_clicks_in_top5[idx])), '{:.3f}'.format(100*np.mean(tfidf_percentage_clicks_in_top5[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_percentage_clicks_in_top5[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_percentage_clicks_in_top5[idx],bm25_percentage_clicks_in_top5[idx],equal_var=False).pvalue))) for idx in range(7)])

In [ ]:
metric_scores.append(['% All Clicks in Top 10 Positions']+[('{:.3f}'.format(100*np.mean(bm25_percentage_clicks_in_top10[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_percentage_clicks_in_top10[idx])), '{:.3f}'.format(100*np.mean(tfidf_percentage_clicks_in_top10[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_percentage_clicks_in_top10[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_percentage_clicks_in_top10[idx],bm25_percentage_clicks_in_top10[idx],equal_var=False).pvalue))) for idx in range(7)])

In [ ]:
metric_scores.append(['% All Clicks in Top 20 Positions']+[('{:.3f}'.format(100*np.mean(bm25_percentage_clicks_in_top20[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_percentage_clicks_in_top20[idx])), '{:.3f}'.format(100*np.mean(tfidf_percentage_clicks_in_top20[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_percentage_clicks_in_top20[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_percentage_clicks_in_top20[idx],bm25_percentage_clicks_in_top20[idx],equal_var=False).pvalue))) for idx in range(7)])

In [ ]:
metric_scores.append(['', '', '', '', '', '', '', ''])

In [ ]:
metric_scores.append(['% Clicks in Top 3 Positions Leading to Insights']+[('{:.3f}'.format(100*np.mean(bm25_percentage_clicks_in_top3_leading_to_insights[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_percentage_clicks_in_top3_leading_to_insights[idx])), '{:.3f}'.format(100*np.mean(tfidf_percentage_clicks_in_top3_leading_to_insights[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_percentage_clicks_in_top3_leading_to_insights[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_percentage_clicks_in_top3_leading_to_insights[idx],bm25_percentage_clicks_in_top3_leading_to_insights[idx],equal_var=False).pvalue))) for idx in range(7)])


In [ ]:
metric_scores.append(['% Clicks in Top 5 Positions Leading to Insights']+[('{:.3f}'.format(100*np.mean(bm25_percentage_clicks_in_top5_leading_to_insights[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_percentage_clicks_in_top5_leading_to_insights[idx])), '{:.3f}'.format(100*np.mean(tfidf_percentage_clicks_in_top5_leading_to_insights[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_percentage_clicks_in_top5_leading_to_insights[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_percentage_clicks_in_top5_leading_to_insights[idx],bm25_percentage_clicks_in_top5_leading_to_insights[idx],equal_var=False).pvalue))) for idx in range(7)])


In [ ]:
metric_scores.append(['% Clicks in Top 10 Positions Leading to Insights']+[('{:.3f}'.format(100*np.mean(bm25_percentage_clicks_in_top10_leading_to_insights[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_percentage_clicks_in_top10_leading_to_insights[idx])), '{:.3f}'.format(100*np.mean(tfidf_percentage_clicks_in_top10_leading_to_insights[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_percentage_clicks_in_top10_leading_to_insights[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_percentage_clicks_in_top10_leading_to_insights[idx],bm25_percentage_clicks_in_top10_leading_to_insights[idx],equal_var=False).pvalue))) for idx in range(7)])


In [ ]:
metric_scores.append(['% Clicks in Top 20 Positions Leading to Insights']+[('{:.3f}'.format(100*np.mean(bm25_percentage_clicks_in_top20_leading_to_insights[idx]))+'±'+'{:.3f}'.format(100*np.std(bm25_percentage_clicks_in_top20_leading_to_insights[idx])), '{:.3f}'.format(100*np.mean(tfidf_percentage_clicks_in_top20_leading_to_insights[idx]))+'±'+'{:.3f}'.format(100*np.std(tfidf_percentage_clicks_in_top20_leading_to_insights[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_percentage_clicks_in_top20_leading_to_insights[idx],bm25_percentage_clicks_in_top20_leading_to_insights[idx],equal_var=False).pvalue))) for idx in range(7)])


In [ ]:
metric_scores.append(['', '', '', '', '', '', '', ''])

In [ ]:
N=5
metric_scores.append([f'nDCG@{N}'])

for idx in range(7):
    bm25_ndcg = [ndcg_score([ndcg_session], [list(range(N))[::-1]]) for ndcg_session in bm25_ndcg_5[idx]]
    tfidf_ndcg = [ndcg_score([ndcg_session], [list(range(N))[::-1]]) for ndcg_session in tfidf_ndcg_5[idx]]

    metric_scores[-1].append(('{:.3f}'.format(np.mean(bm25_ndcg))+'±'+'{:.3f}'.format(np.std(bm25_ndcg)), '{:.3f}'.format(np.mean(tfidf_ndcg))+'±'+'{:.3f}'.format(np.std(tfidf_ndcg)), float('{:.4f}'.format(stats.ttest_ind(tfidf_ndcg,bm25_ndcg,equal_var=False).pvalue))))


In [ ]:
N=10
metric_scores.append([f'nDCG@{N}'])

for idx in range(7):
    bm25_ndcg = [ndcg_score([ndcg_session], [list(range(N))[::-1]]) for ndcg_session in bm25_ndcg_10[idx]]
    tfidf_ndcg = [ndcg_score([ndcg_session], [list(range(N))[::-1]]) for ndcg_session in tfidf_ndcg_10[idx]]

    metric_scores[-1].append(('{:.3f}'.format(np.mean(bm25_ndcg))+'±'+'{:.3f}'.format(np.std(bm25_ndcg)), '{:.3f}'.format(np.mean(tfidf_ndcg))+'±'+'{:.3f}'.format(np.std(tfidf_ndcg)), float('{:.4f}'.format(stats.ttest_ind(tfidf_ndcg,bm25_ndcg,equal_var=False).pvalue))))


In [ ]:
N=20
metric_scores.append([f'nDCG@{N}'])

for idx in range(7):
    bm25_ndcg = [ndcg_score([ndcg_session], [list(range(N))[::-1]]) for ndcg_session in bm25_ndcg_20[idx]]
    tfidf_ndcg = [ndcg_score([ndcg_session], [list(range(N))[::-1]]) for ndcg_session in tfidf_ndcg_20[idx]]

    metric_scores[-1].append(('{:.3f}'.format(np.mean(bm25_ndcg))+'±'+'{:.3f}'.format(np.std(bm25_ndcg)), '{:.3f}'.format(np.mean(tfidf_ndcg))+'±'+'{:.3f}'.format(np.std(tfidf_ndcg)), float('{:.4f}'.format(stats.ttest_ind(tfidf_ndcg,bm25_ndcg,equal_var=False).pvalue))))


In [ ]:
metric_scores.append(['', '', '', '', '', '', '', ''])

In [ ]:
metric_scores.append(['Mean Average Precision']+[('{:.3f}'.format(np.mean(bm25_average_precision[idx]))+'±'+'{:.3f}'.format(np.std(bm25_average_precision[idx])), '{:.3f}'.format(np.mean(tfidf_average_precision[idx]))+'±'+'{:.3f}'.format(np.std(tfidf_average_precision[idx])), float('{:.4f}'.format(stats.ttest_ind(tfidf_average_precision[idx],bm25_average_precision[idx],equal_var=False).pvalue))) for idx in range(7)])

In [ ]:
metric_scores_df = pd.DataFrame(metric_scores, columns = ['Metric', '1', '2-3', '4-5', '6-10', '11-20', '21-inf', 'ALL'])
metric_scores_df.to_csv('running_avg_NEW_FORMAT.csv', index=False)

In [ ]:
#ignore this

from math import sqrt
from statsmodels.stats.power import TTestIndPower
  
#calculation of effect size
# size of samples in pilot study
n1, n2 = 13002, 49790
  
# variance of samples in pilot study
s1, s2 = 12.353**2, 12.295**2
  
# calculate the pooled standard deviation 
# (Cohen's d)
s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
  
# calculate the effect size
d = (0.206) / s
print(f'Effect size: {d}')
  
# factors for power analysis
alpha = 0.1
power = 0.8
  
# perform power analysis to find sample size 
# for given effect
obj = TTestIndPower()
n = obj.solve_power(effect_size=d, alpha=alpha, power=power, 
                    ratio=1, alternative='two-sided')
  
print('Sample size/Number needed in each group: {:.3f}'.format(n))

In [ ]:
from statsmodels.stats.power import TTestIndPower
  

power = TTestIndPower()
n_test = power.solve_power(nobs1=n1/2+n2/2, effect_size = d,
                           power = None, alpha = 0.05, ratio=1)

In [ ]:
n_test

In [ ]:
n1/2+n2/2